In [1]:
import h5py
import importlib
import os
import json
import shutil
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import tensorflow as tf
import yaml
import logomaker
import time
import tfomics
from tfomics import impress, explain, moana

from hominid_pipeline import utils, model_zoo, hominid, layers
import keras

2023-06-12 11:55:09.947073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 11:55:10.782572: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-12 11:55:10.782644: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-12 11:55:10.782650: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [14]:
def absmaxND(a, axis=None):
    amax = np.max(a, axis)
    amin = np.min(a, axis)
    return np.where(-amin > amax, amin, amax)


def get_layer_output(model, index, X):
    temp = tf.keras.Model(inputs=model.inputs, outputs=model.layers[index].output)
    return temp.predict(X)

def pearsonr(vector1, vector2):
    m1 = np.mean(vector1)
    m2 = np.mean(vector2)
    
    diff1 = vector1 - m1
    diff2 = vector2 - m2
    
    top = np.sum(diff1 * diff2)
    bottom = np.sum(np.power(diff1, 2)) * np.sum(np.power(diff2, 2))
    bottom = np.sqrt(bottom)
    
    return top/bottom




def plot_glifac(ax, correlation_matrix, filter_labels, vmin=-0.5, vmax=0.5):
    ax.set_xticks(list(range(len(filter_labels))))
    ax.set_yticks(list(range(len(filter_labels))))
    ax.set_xticklabels(filter_labels, rotation=90)
    ax.set_yticklabels(filter_labels)
    c = ax.imshow(correlation_matrix, cmap='bwr_r', vmin=vmin, vmax=vmax)
    return ax, c


def correlation_matrix(model, c_index, mha_index, X, thresh=0.1, random_frac=0.5, limit=None, head_concat=np.max, symmetrize=absmaxND):
    
    """
    * model                  trained tensorflow model
    * c_index                index of the convolutoinal layer (after pooling)
    * mha_index              index of multi-head attention layer
    * X                      test sequences
    * thresh                 attention threshold
    * random_frac            proportion of negative positions in the set of position interactions
    * limit                  maximum number of position interactions processed; sometimes needed to avoid resource exhaustion
    * head_concat            function for concatenating heads; e.g. np.max, np.mean
    * symmetrize             function for symmetrizing the correlation matrix across diagonal
    """
    
    assert 0 <= random_frac < 1
    
    feature_maps = get_layer_output(model, c_index, X)
    o, att_maps = get_layer_output(model, mha_index, X)
    att_maps = head_concat(att_maps, axis=1)
    
    position_interactions = get_position_interactions(att_maps, thresh)
    num_rands = int(random_frac/(1-random_frac))
    random_interactions = [np.random.randint(len(att_maps), size=(num_rands, 1)), np.random.randint(att_maps.shape[1], size=(num_rands, 2))]
    position_pairs = [np.vstack([position_interactions[0], random_interactions[0]]), np.vstack([position_interactions[1], random_interactions[1]])]
    if limit is not None:
        permutation = np.random.permutation(len(position_pairs[0]))
        position_pairs = [position_pairs[0][permutation], position_pairs[1][permutation]]
        position_pairs = [position_pairs[0][:limit], position_pairs[1][:limit]]
    
    filter_interactions = feature_maps[position_pairs].transpose([1, 2, 0])
    correlation_matrix = correlation(filter_interactions[0], filter_interactions[1])
    if symmetrize is not None:
        correlation_matrix = symmetrize(np.array([correlation_matrix, correlation_matrix.transpose()]), axis=0)
    correlation_matrix = np.nan_to_num(correlation_matrix)
    
    return correlation_matrix

    
def get_position_interactions(att_maps, threshold=0.1):
    position_interactions = np.array(np.where(att_maps >= threshold))
    position_interactions = [position_interactions[[0]].transpose(), position_interactions[[1, 2]].transpose()]
    return position_interactions
    
    
def correlation(set1, set2, function=pearsonr):
    combinations = np.indices(dimensions=(set1.shape[0], set2.shape[0])).transpose().reshape((-1, 2)).transpose()[::-1]
    vector_mesh = [set1[combinations[0]], set2[combinations[1]]]
    vector_mesh = np.array(vector_mesh).transpose([1, 0, 2])
    correlations = []
    for i in range(len(vector_mesh)):
        r = function(vector_mesh[i][0], vector_mesh[i][1])
        correlations.append(r)
    correlations = np.array(correlations).reshape((len(set1), len(set2)))
    return correlations

In [15]:
working_dir = "/home/chandana/projects/hominid_pipeline/results"
hits = [
    "experiments/sweeps/tune_hominid_8f34a_00085_85_conv1_activation=exponential,conv1_attention_pool_size=23,conv1_batchnorm=True,conv1_channel_weight=se_2023-05-18_00-04-12",
    "experiments/sweeps/tune_hominid_8f34a_00185_185_conv1_activation=relu,conv1_attention_pool_size=23,conv1_batchnorm=True,conv1_channel_weight=softconv_2023-05-18_06-11-38",
    "experiments/model_variations/tune_hominid_8f34a_00058_58_conv1_activation=relu,conv1_attention_pool_size=30,conv1_batchnorm=False,conv1_channel_weight=softconv_2023-05-17_22-09-59/exponential", # this one!
    "experiments/model_variations/tune_hominid_8f34a_00056_56_conv1_activation=relu,conv1_attention_pool_size=5,conv1_batchnorm=False,conv1_channel_weight=se,conv1__2023-05-17_22-05-53/exponential",
    "experiments/model_variations/tune_hominid_8f34a_00058_58_conv1_activation=relu,conv1_attention_pool_size=30,conv1_batchnorm=False,conv1_channel_weight=softconv_2023-05-17_22-09-59/exponential/variations/variation_1"    
]

In [16]:
index = 1
save_path = f"{working_dir}/{hits[index]}"
config_file = f"{working_dir}/{hits[index]}/config.yaml"
config = hominid.load_config(config_file)

tuner = hominid.HominidTuner(
    config, 
    epochs=100, 
    tuning_mode=False, 
    save_path=save_path, 
    subsample=False
)

tuner.save_path

'/home/chandana/projects/hominid_pipeline/results/experiments/sweeps/tune_hominid_8f34a_00185_185_conv1_activation=relu,conv1_attention_pool_size=23,conv1_batchnorm=True,conv1_channel_weight=softconv_2023-05-18_06-11-38'

In [18]:
print(f"Loading model and dataset!")

x_test, y_test = tuner.data_processor.load_data("test")

# Build the model
model = tuner.model_builder.build_model()

model.compile(
    tf.keras.optimizers.Adam(lr=0.001),
    loss='mse',
    metrics=[utils.Spearman, utils.pearson_r]
    )
print(model.summary())
model.load_weights(f'{tuner.save_path}/weights')

Loading model and dataset!
Building model...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 249, 4)]     0           []                               
                                                                                                  
 conv1 (Conv1D)                 (None, 249, 96)      7392        ['input[0][0]']                  
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 249, 96)      384         ['conv1[0][0]']                  
                                                                                                  
 conv1_activation (Activation)  (None, 249, 96)      0           ['conv1_bn[0][0]']               
                                                                                              

In [20]:
# for this model ONLY: challenges are that there is no pooling layer
# so selected the concatenation layer for the conv_layer

t1 = time.time()
sample = x_test[:5000]
lays = [type(i) for i in model.layers]
c_index = lays.index(tf.keras.layers.MaxPool1D) # lays.index(keras.layers.core.tf_op_layer.TFOpLambda) #

mha_index = lays.index(layers.MultiHeadAttention)
correlation_map = correlation_matrix(
                            model, 
                            c_index, 
                            mha_index, 
                            sample, 
                            thresh=0.1, 
                            random_frac=0.3, 
                            limit=150000
                        )
t2 = time.time()

print(f"Time taken: {t2-t1}")

157/157 [==============================] - 1s 3ms/step


/home/chandana/miniforge3/envs/deepstarr/lib/python3.7/site-packages/ipykernel_launcher.py:65: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/chandana/miniforge3/envs/deepstarr/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in float_scalars


Time taken: 129.41733574867249


In [28]:
import numpy as np
import tensorflow as tf


def absmaxND(a, axis=None):
    amax = np.max(a, axis)
    amin = np.min(a, axis)
    return np.where(-amin > amax, amin, amax)


def get_layer_output(model, index, X):
    temp = tf.keras.Model(inputs=model.inputs, outputs=model.layers[index].output)
    return temp.predict(X)


def pearsonr(vector1, vector2):
    diff1 = vector1 - np.mean(vector1)
    diff2 = vector2 - np.mean(vector2)
    top = np.sum(diff1 * diff2)
    bottom = np.sqrt(np.sum(diff1 ** 2) * np.sum(diff2 ** 2))
    return top / bottom


def plot_glifac(ax, correlation_matrix, filter_labels, vmin=-0.5, vmax=0.5):
    ax.set_xticks(np.arange(len(filter_labels)))
    ax.set_yticks(np.arange(len(filter_labels)))
    ax.set_xticklabels(filter_labels, rotation=90)
    ax.set_yticklabels(filter_labels)
    c = ax.imshow(correlation_matrix, cmap='bwr_r', vmin=vmin, vmax=vmax)
    return ax, c

In [38]:


def _correlation_matrix(model, c_index, mha_index, X, thresh=0.1, random_frac=0.5, limit=None, head_concat=np.max,
                       symmetrize=absmaxND):
    assert 0 <= random_frac < 1

    feature_maps = get_layer_output(model, c_index, X)
    o, att_maps = get_layer_output(model, mha_index, X)
    att_maps = head_concat(att_maps, axis=1)

    position_interactions = _get_position_interactions(att_maps, thresh)
    
#     num_rands = int(random_frac / (1 - random_frac))
#     random_interactions = [np.random.randint(len(att_maps), size=num_rands),
#                            np.random.randint(att_maps.shape[1], size=(num_rands, 2))]
    
    
#     print(num_rands)
    
#     position_pairs = [np.vstack([position_interactions[0], random_interactions[0]]),
#                       np.vstack([position_interactions[1], random_interactions[1]])]
#     if limit is not None:
#         permutation = np.random.permutation(len(position_pairs[0]))
#         position_pairs = [position_pairs[0][permutation], position_pairs[1][permutation]]
#         position_pairs = [position_pairs[0][:limit], position_pairs[1][:limit]]

#     filter_interactions = feature_maps[position_pairs].transpose([1, 2, 0])
#     correlation_matrix = _correlation(filter_interactions[0], filter_interactions[1], pearsonr)
#     if symmetrize is not None:
#         correlation_matrix = symmetrize(np.array([correlation_matrix, correlation_matrix.transpose()]), axis=0)
#     correlation_matrix = np.nan_to_num(correlation_matrix)

#     return correlation_matrix
    return 0


def _get_position_interactions(att_maps, threshold=0.1):
    position_interactions = np.argwhere(att_maps >= threshold)
    position_interactions = [position_interactions[:, 0], position_interactions[:, 1:]]
    return position_interactions


def _correlation(set1, set2, function=pearsonr):
    combinations = np.indices((set1.shape[0], set2.shape[0])).reshape((-1, 2))[:, ::-1]
    vector_mesh = [set1[combinations[:, 0]], set2[combinations[:, 1]]]
    correlations = np.array([function(vector_mesh[i][0], vector_mesh[i][1]) for i in range(len(vector_mesh))])
    correlations = correlations.reshape((len(set1), len(set2)))
    return correlations

In [41]:
num_rands = int(random_frac/(1-random_frac))
num_rands
random_interactions = [np.random.randint(len(att_maps), size=(num_rands, 1)), 
                       np.random.randint(att_maps.shape[1], size=(num_rands, 2))]
# position_pairs = [np.vstack([position_interactions[0], random_interactions[0]]), np.vstack([position_interactions[1], random_interactions[1]])]

1

In [39]:
# for this model ONLY: challenges are that there is no pooling layer
# so selected the concatenation layer for the conv_layer

t1 = time.time()
sample = x_test[:5000]
lays = [type(i) for i in model.layers]
c_index = lays.index(tf.keras.layers.MaxPool1D) # lays.index(keras.layers.core.tf_op_layer.TFOpLambda) #

mha_index = lays.index(layers.MultiHeadAttention)
correlation_map_2 = _correlation_matrix(
                            model, 
                            c_index, 
                            mha_index, 
                            sample, 
                            thresh=0.1, 
                            random_frac=0.3, 
                            limit=150000
                        )
t2 = time.time()

print(f"Time taken: {t2-t1}")

157/157 [==============================] - 1s 3ms/step
0
Time taken: 1.400761604309082
